Introduction
 

Purpose of notebook: Hackathon Womanium Quantinuum (QNLP)

Date of development: August 2022

Developed by: Tessa Evers

Note!: This is a draft, many actions can be improved, please do so!

In [ ]:
# pip install pandas
# pip install nltk
# pip install -U scikit-learn

In [4]:
# Import packages
import numpy as np
import nltk
# nltk.download('punkt')
import pandas as pd
import os
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import blankline_tokenize 
from tqdm.autonotebook import tqdm
from nltk.util import bigrams, trigrams, ngrams
from sklearn.model_selection import train_test_split

In [2]:
from lambeq import BobcatParser
from discopy import grammar

In [3]:
#working directory
print(os.getcwd())
os.chdir('/home/jovyan/Quantum-Natural-Language-Processing-with-lambeq---Quantinuum/')
print(os.getcwd())

/home/jovyan/Tessa
/home/jovyan/Quantum-Natural-Language-Processing-with-lambeq---Quantinuum


## Task 2: Get an overview of the dataset

Actions Task 2
- familiarise yourself with the dataset
- learning objective of the model = write, train, test a simple QNLP model that can detect wether they belong to the same category (both IT, or both Food)(y = 1) or not (y = 0).
- preprocessing the data with classical NLTK package for example
- split data in training and test sets


(FYI: later on we will (1) find a suitable quantum representation (2) check the similarity between the sentences)

In [5]:
# Import data Quantinuum
df_qn = pd.read_csv('MC1.TXT', sep = ",", names =["themei", "themej", "y"])

In [6]:
# The data (pair of sentences) we use to predict y 
df_qn['sentence_pair'] = df_qn['themei'] + ", " + df_qn['themej']

In [7]:
df_qn

,themei,themej,y,sentence_pair
0,cook creates complicated dish,experienced chef prepares complicated dish,1,"cook creates complicated dish, experienced ch..."
1,skilful programmer creates code,devoted hacker writes code,1,"skilful programmer creates code, devoted hack..."
2,skilful cook creates meal,devoted hacker creates complicated code,0,"skilful cook creates meal, devoted hacker cre..."
3,hacker writes code,skilful hacker creates code,1,"hacker writes code, skilful hacker creates code"
4,devoted hacker writes code,hacker writes complicated code,1,"devoted hacker writes code, hacker writes com..."
...,...,...,...,...
95,skilful programmer writes advanced code,skilful cook prepares meal,0,"skilful programmer writes advanced code, skil..."
96,programmer creates code,chef creates dish,0,"programmer creates code, chef creates dish"
97,hacker creates code,chef prepares dish,0,"hacker creates code, chef prepares dish"
98,skilful programmer creates complicated code,programmer writes complicated code,1,"skilful programmer creates complicated code, ..."


In [ ]:
## Draft for tokenizing
# rows = []
# for fr, row in df_qn[['sentence_pair']].iterrows():
#     rowa = np.array(word_tokenize(df_qn.loc[fr,'sentence_pair']))
#     rowab = ' '.join(map(str,rowa))
#     rows.append(rowab)
#     # df_qn.loc[fr, 'sentence_pair_tok'] = word_tokenize(df_qn.loc[fr,'sentence_pair'])

# df_qn['sentence_pair_tok'] = rows#pd.DataFrame(rows, columns= ["sentence_pair_tok"]).copy()

## Preprocessing data

Actions: 
- Lemmatize
- Tokenize
- ..... have a look what else is possible by using for example NLTK!

In [ ]:
# Tokenizing by seperating the pair and later combining
# Open action: create a function of this!
rowsi = []
rowsj = []
for fr, row in df_qn[['themei']].iterrows():
    rowai = np.array(word_tokenize(df_qn.loc[fr,'themei']))
    rowaj = np.array(word_tokenize(df_qn.loc[fr,'themej']))
    rowabi = ' '.join(map(str,rowai))
    rowabj = ' '.join(map(str,rowaj))
    rowsi.append(rowabi)
    rowsj.append(rowabj)

df_qn['sentence_pair_toki'] = rowsi 
df_qn['sentence_pair_tokj'] = rowsj
df_qn['sentence_pair_tok'] = df_qn['sentence_pair_toki'] + ", " + df_qn['sentence_pair_tokj'] 

In [ ]:
# Open action: make sure to take into account the "," so ADAPT
# Open action: create a function of this!
rowsi = []
rowsj = []
for fr, row in df_qn[['sentence_pair_toki']].iterrows():
    bigrami = list(nltk.bigrams(df_qn.loc[fr,'sentence_pair_toki'].split(" ")))
    bigramj = list(nltk.bigrams(df_qn.loc[fr,'sentence_pair_tokj'].split(" ")))
    rowai = np.array(bigrami)
    rowaj = np.array(bigramj)
    rowabi = ' '.join(map(str,rowai))
    rowabj = ' '.join(map(str,rowaj))
    rowsi.append(rowabi)
    rowsj.append(rowabj)

df_qn['sentence_pair_tokbi'] = rowsi 
df_qn['sentence_pair_tokbj'] = rowsj
df_qn['sentence_pair_tokb'] = df_qn['sentence_pair_tokbi'] + ", " + df_qn['sentence_pair_tokbj'] 


In [ ]:
df_qn

In [ ]:
df_qn_pair = pd.DataFrame()
df_qn_pair['sentence_pair'] = df_qn['sentence_pair']
# df_qn_pair['sentence_pair_tok'] = df_qn['sentence_pair_tok']
df_qn_pair['y'] = df_qn['y']

In [ ]:
df_qn_pair

In [ ]:
sentence = df_qn_pair.loc[0, 'sentence_pair']
print(sentence)
print(type(sentence))

## How does the data look like?

Have a look at the distribution of the words in the different sentences

In [ ]:
# test frequence of words
fdist = FreqDist()

for word in sentence.split(" "):
    fdist[word.lower()]+=1
    
print(fdist)

In [ ]:
print(sentence)
print(fdist['cook'])
fdist.most_common()

In [ ]:
# test frequence of words
fdist_complete = FreqDist()

for i, row in df_qn_pair[['sentence_pair_tok']].iterrows():
    rowad = df_qn_pair.loc[i,'sentence_pair_tok']
    for word in rowad.split(" "):
        fdist_complete[word.lower()]+=1
    
print(fdist_complete)

In [ ]:
fdist_complete.most_common()

In [ ]:
# test frequence of words
fdist_complete_notok = FreqDist()

for i, row in df_qn_pair[['sentence_pair']].iterrows():
    rowad = df_qn_pair.loc[i,'sentence_pair']
    for word in rowad.split(" "):
        fdist_complete_notok[word.lower()]+=1
    
print(fdist_complete_notok)

In [ ]:
fdist_complete_notok.most_common()

## Make it ready as input for training the model in furter tasks

In [8]:
# Split the data for training and test purposes
X_train, X_test, y_train, y_test = train_test_split(df_qn['sentence_pair'], df_qn['y'], test_size=0.33, random_state=42)

## Task 3: Parse sentences to diagrams

* Investigate the readers
* Make sure that parsing is done correctly by investigating the Rewriters
* Identify a selection process of how to determine which reader will be used

## DisCoCat with BobcatParser

*explain here* 
1. how it works 
2. what are the advantages
3. what are the disadvantages
4. can we use other parsers?

In [ ]:
# Parse the sentence and convert it into a string diagram
parser = BobcatParser(verbose='suppress')
print(parser)
print(type(parser))

In [ ]:
sentence

In [ ]:
diagram = parser.sentence2diagram(sentence)
print(diagram)
print(type(diagram))

In [ ]:
#string diagram, given a compositional scheme
grammar.draw(diagram, figsize=(11,3), fontsize=12)

## Different types of readers

## Spiders_reader 

*explain here* 
1. how it works 
2. what are the advantages
3. what are the disadvantages
4. when to select and when to prefer using another reader

First note: the spiders_reader seems much faster (for me specific)

In [ ]:
from lambeq import spiders_reader

In [ ]:
# Create string diagrams based on spiders reader
spiders_diagram = spiders_reader.sentence2diagram(sentence)
print(spiders_diagram)

In [ ]:
# Not a pregroup diagram, we can't use grammar.draw()
spiders_diagram.draw(figsize=(13,6), fontsize=12)

## Cups_reader 

*explain here* 
1. how it works 
2. what are the advantages
3. what are the disadvantages
4. when to select and when to prefer using another reader

First note: the cups_reader seems also quite fast

In [ ]:
from lambeq import cups_reader

# Create string diagrams based on cups reader
cups_diagram = cups_reader.sentence2diagram(sentence)

In [ ]:
grammar.draw(cups_diagram, figsize=(12,3), fontsize=12)

## Stairs_reader

*explain here* 
1. how it works 
2. what are the advantages
3. what are the disadvantages
4. when to select and when to prefer using another reader


In [ ]:
from lambeq import stairs_reader

stairs_diagram = stairs_reader.sentence2diagram(sentence)

In [ ]:
stairs_diagram.draw(figsize=(12,5), fontsize=12)

## TreeReader

*explain here* 
1. how it works 
2. what are the advantages
3. what are the disadvantages
4. when to select and when to prefer using another reader


In [ ]:
from lambeq import TreeReader

reader = TreeReader()
tree_diagram = reader.sentence2diagram(sentence)

In [ ]:
tree_diagram.draw(figsize=(12,5), fontsize=12)

In [ ]:
from lambeq import TreeReader, TreeReaderMode

reader = TreeReader(mode=TreeReaderMode.RULE_ONLY)
tree_diagram = reader.sentence2diagram(sentence)

In [ ]:
tree_diagram.draw(figsize=(12,5), fontsize=12)

## How do we rewrite the diagrams when connections are wrong?



## Task 4: From diagram to circuit
Goal: prepare the quantum circuits for each sentence in your dataset. 
We use an ansatz that 
1. Maps each wire to a qubit system.
2. Maps each box to a variational quantum circuit.

Actions
- Use IQPAnsatz
- Try different number of qubits
- Try differrent ansatz layers
- Look if hyperparameter tuning can be use like in classical computing when training classified models f.e.!

## Task 5: Define the learning objective
Goal: prepare the quantaum circuits for each sentence in your dataset

Actions
- Use first NumpyModel. How does it work? Can we improve?
- What similarity metrics are already available and which do work best to predict the similarity between sentences?
- Create a working experiment (recall this from working with Qiskit!)
- When the model works well, apply it on a simulator!

Note: for more tips look at the assignment description!

## Task 6: Run your first QNLP experiment

Actions

Note: for more tips look at the assignment description!

## Notes / rest

In [ ]:
# from qiskit import *
# from qiskit.visualization import plot_histogram
# from qiskit.tools.monitor import job_monitor

# bell = QuantumCircuit(2, 2)
# bell.h(0)
# bell.cx(0, 1)

# meas = QuantumCircuit(2, 2)
# meas.measure([0,1], [0,1])

# backend = BasicAer.get_backend('qasm_simulator')
# circ = bell + meas
# result = execute(circ, backend, shots=1000).result()
# counts  = result.get_counts(circ)
# print(counts)

# plot_histogram(counts)